# MODEL DE SEGMENTATION D'IMAGES (pour la prédiction)

Ce script présente un modèle pour segmenter les images RGB. Il est composé de 4 parties :

1- Importation des librairies

2- Importation de l'image au format RGB [Input]

3- Traitement de l'image par le modèle segmentation [Processing]

4- Exporation de l'image segmentée au format binaire [Output]

En particulier l'étape de traitement (étape 3) est composée en trois sous-parties :

i) Redimension de l'image en 256 x 256 pixel

ii) Segmentation de l'image par l'algorithme U-NET

iii) Application du seuil de segmentation optimisé sur l'image

Structure du répertoire : 

- Répertoire :
    - segmentation_model_prediciton.ipynb
    - best_model_FL_BCE_0_5.h5
    - input :
        - sample1.jpg
        - sample2.jpg
    - output :
        - output_sample1.jpg
        - output_sample2.jpg

## Importation des librairies

Librairies manquantes : executer le code suivant 

!pip install librairie

In [ ]:
!pip install focal_loss

Importation des librairies

In [24]:
import sys
import numpy
import tensorflow as tf
from tensorflow import keras
from focal_loss import BinaryFocalLoss
from tensorflow.keras import backend as K
import numpy as np
import cv2
from skimage.io import imread, imshow, imread_collection, concatenate_images
from skimage.transform import resize
import os
from tqdm import tqdm
from skimage.io import imread

## Fonctions

La fonction redimension() prends en entrée une image RGB et retourne quatre objets :

- X : l'image au dimension 256 x 256 pixel

- size_ : taille original de l'image

In [30]:
def redimension(image):
    X = np.zeros((1,256,256,3),dtype=np.uint8)
    #img = cv2.imread(image,cv2.COLOR_BGR2RGB)
    img = imread(image)[:,:,:3]
    size_ = img.shape
    img = img[:,:,:3]
    X[0] = resize(img, (256, 256), mode='constant', preserve_range=True)
    return X,size_

def dice_coefficient(y_true, y_pred):
    eps = 1e-6
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection) / (K.sum(y_true_f * y_true_f) + K.sum(y_pred_f * y_pred_f) + eps) #eps pour éviter la division par 0 

## Algorithme

### Input

Prends les images à prédire à partir du dossier input
L'utilisateur doit penser à ranger ses images dans le dossier input

In [31]:
image_sample = os.listdir("./input")
print("Input done")

Input done


### Redimension en 256 x 256 pixels

In [32]:
image_sample_reshaped = []
size_original = []

for sample in tqdm(image_sample, total = len(image_sample),
              desc ="Reshaped..."):
    image_reshaped,size_ = redimension("./input/"+sample)
    image_sample_reshaped.append(image_reshaped)
    x,y,z = size_
    size_original.append((x,y))
print("Reshaped done")

Reshaped...: 100%|██████████████████████████████████████████████████████████████████████████| 7/7 [00:11<00:00,  1.58s/it]

Reshaped done


### Segmentation de l'image par l'algorithme U-NET pré-entrainé

In [33]:
model_new = tf.keras.models.load_model("best_model_FL_BCE_0_5.h5",custom_objects={'dice_coefficient': dice_coefficient})
print("Model imported\n")

image_prediction = []
for sample in image_sample_reshaped:
    prediction = model_new.predict(sample)
    
    #Application du seuil de segmentation optimisé
    preds_test_t = (prediction > 0.3)
    image_prediction.append(preds_test_t)
print("Prediction done\n")

Model imported

Prediction done



### Output

Enregistre les images segmentés dans le dossier output sous le nom output_*nom de l'image*

In [34]:
image_output = []
for sample,i,name in zip(image_prediction,range(len(size_original)),image_sample):
    preds_test_t = resize(sample[0,:,:,0], size_original[i], mode='constant', preserve_range=True)
    cv2.imwrite("./output/output_"+name,preds_test_t*255)
print("Exportation done")

Exportation done
